In [2]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from tqdm import tqdm

from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline
from lifelines import CoxPHFitter
import shap
import xgboost
import matplotlib.pyplot as plt
import statistics as st
from bayes_opt import BayesianOptimization
from pyirr import intraclass_correlation
from lightgbm import LGBMClassifier
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [3]:
from feature_utils.feature_classfication import data_normalization, data_normalization_apply_cohort1_to_all, lgb_evaluate_lgbm, lgb_evaluate_svm

In [ ]:
data = pd.read_csv('/Users/wangpengcheng/Documents/qsu_data/sub_all_n4b_c.csv')
data = data[(data['Subtypes_three']==0)|(data['Subtypes_three']==1)|(data['Subtypes_three']==2)]
id_all = data[(data['Cohort']==1)]['id']
id_ex = data[(data['Cohort']!=1)]['id']

data.reset_index(inplace=True)

for i in range(data.shape[0]):
    if data.loc[i, 'Sex'] == 'f' or data.loc[i, 'Sex'] == 'F':
        data.loc[i, 'Sex'] = 0
    else:
        data.loc[i, 'Sex'] = 1
        
# delete rows with nan
data = data.dropna(axis=0, subset=['T2_diagnostics_Image-original_Mean_x'])
data.reset_index(inplace=True)

label = data['Subtypes_three']
cohort = data['Cohort']
age = data['Age']

data = data.iloc[:, np.r_[8:7896]]
data = data.dropna(axis=1,how='any')

for col in data.columns:
    try:
        data[col] = data[col].astype(float)
    except:
        del data[col]

data = data_normalization(data)
data = pd.concat([cohort, label, age, data], axis=1, ignore_index=False)

print("subjects: {}, features: {}".format(data.shape[0],data.shape[1]-2))
print("Head 8 columns name:", data.columns[:8])

In [ ]:
# do feature selection
from feature_utils.feature_selection import feature_selection_rf, feature_selection_auc
from collections import Counter

cohort1_data = data[data['Cohort']==1].iloc[:, 2:]
cohort1_label = data[data['Cohort']==1].iloc[:, 1]

# random forest selection
features_all_sel = []
for m in tqdm(range(100)):
    sel_data = feature_selection_rf(cohort1_data, cohort1_label, feature_num=20)
    features_all_sel = features_all_sel + sel_data.columns.tolist()
print(Counter(features_all_sel))

# AUC selection
sel_data = feature_selection_auc(cohort1_data, cohort1_label, range_auc=[0.42, 0.58])

In [ ]:
# selected features
wnt_shh_g34_feature_names = ['T1E_log-sigma-3-mm-3D_gldm_LargeDependenceHighGrayLevelEmphasis_x',
 'T1E_log-sigma-3-mm-3D_glcm_Autocorrelation_x',
 'T1E_log-sigma-1-mm-3D_glcm_ClusterShade_x',
 'T1E_wavelet-LHL_firstorder_Mean_x',
 'T2_wavelet-LLH_glcm_ClusterShade_y',
 'T1E_log-sigma-5-mm-3D_glrlm_LowGrayLevelRunEmphasis_x',
 'T1E_wavelet-HLL_firstorder_Median_x',
 'T1E_wavelet-LHL_firstorder_Median_x',
 'T2_wavelet-HHL_firstorder_Skewness_x',
 'T1E_log-sigma-3-mm-3D_glrlm_LongRunLowGrayLevelEmphasis_x',
 'T1E_lbp-2D_glrlm_RunLengthNonUniformity_x',
 'T2_log-sigma-1-mm-3D_gldm_LowGrayLevelEmphasis_x',
 'T1E_log-sigma-5-mm-3D_firstorder_90Percentile_x',
 'T1E_log-sigma-3-mm-3D_glszm_LowGrayLevelZoneEmphasis_x',
 'T2_wavelet-HHL_glcm_SumSquares_x',
 'T2_wavelet-HHL_glrlm_GrayLevelNonUniformityNormalized_x',
 'T2_wavelet-LLH_gldm_DependenceNonUniformity_x',
 'T2_original_shape_Maximum3DDiameter_y',
 'T2_original_glszm_LargeAreaEmphasis_y']

sel_columns = ['Cohort', 'Subtypes_three', 'Age'] + wnt_shh_g34_feature_names

x = data[sel_columns]
x.iloc[:, :] = data_normalization_apply_cohort1_to_all(x.iloc[:, :])
print(x.shape)

In [ ]:
# split base dataset and external validation dataset
df_val = x[x['Cohort'] != 1]
df_val.reset_index(inplace=True)
X_val = df_val.iloc[:, 3:]

feature_names = X_val.columns.tolist()
X_val = np.array(X_val)
y_val = df_val['Subtypes_three']
y_val = np.array(y_val)

print('X-val: ', X_val.shape)
      
df_X = x[x['Cohort'] == 1]
df_X.reset_index(inplace=True)
X = df_X.iloc[:, 3:]
y = df_X['Subtypes_three']
print('X: ', X.shape)

In [ ]:
# train and find the best hyperparameters for lgbm
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from functools import partial


def f1_scores(label_f1, pred_f1, pos_index):
    """
    Calculate f1 score（for bootstrap）
    :param data: data[:, 0] = pred, data[:, 1] = label, data[:, 2] = index
    :return: f1 score
    """
    conf_mat = confusion_matrix(y_true=label_f1, y_pred=pred_f1)

    tp = conf_mat[pos_index][pos_index]
    fp = np.sum(conf_mat[:, pos_index]) - tp
    fn = np.sum(conf_mat[pos_index]) - tp
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1_cls = 2 * precision * recall / (precision + recall)
    return f1_cls

kf = KFold(n_splits=3, shuffle=True, random_state=32)

for train_index, test_index in kf.split(X):
    X_train, y_train = X.iloc[train_index], y.iloc[train_index]
    X_test, y_test = X.iloc[test_index], y.iloc[test_index]

    X_train = np.array(X_train)
    X_test = np.array(X_test)

    y_train = np.array(y_train).astype(int)
    y_test = np.array(y_test).astype(int)
    
    nm = SMOTE(random_state=42)
    X_res, y_res = nm.fit_resample(X_train, y_train)

    print('size of data partitions')
    print('...............INPUT..............OUTPUT..........')
    print('Train      : ', X_train.shape, '         ', y_train.shape, np.sum(y_train))
    print('Test       : ', X_test.shape, '         ', y_test.shape, np.sum(y_test))
    print('Validation : ', X_val.shape, '         ', y_val.shape, np.sum(y_val))

    # bayesian optimization of hyperparameters for lg_boost
    gp_params = {"alpha": 1e-4}
    seed = 1

    # use partial to pass train, val and test data
    partial_lgb_evaluate_lgbm = partial(lgb_evaluate_lgbm, X_train=X_res, y_train=y_res, X_val=X_val, y_val=y_val,
                                        X_test=X_test, y_test=y_test, seed=1, multi_cls=True)
    lgb_bo = BayesianOptimization(partial_lgb_evaluate_lgbm, {'max_depth': (1, 10),
                                                 'num_leaves': (2,10),
                                                 'learning_rate':(0.01, 0.4),
                                                 'max_bin':(200, 800),
                                                 'colsample_bytree': (0.01, 1.0),
                                                 'reg_alpha':(1,10),
                                                 'reg_lambda':(0.01,1),
                                                 'subsample':(0.01,1),
                                                 'min_child_samples' : (5, 30),
                                                 'min_child_weight':(1, 5),
                                                 'n_estimators' :(500,2200),
                                                 'scale_pos_weight': (1,7),})

    # Optimally needs quite a few more initiation points and number of iterations
    lgb_bo.maximize(init_points=200, n_iter=50, acq='ei',  **gp_params)

    print(lgb_bo.max)

In [19]:
# test and evaluation
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score, roc_auc_score, roc_curve, auc

kf = KFold(n_splits=3, shuffle=True, random_state=32)
test_folds = []
val_folds = []
true_folds = []
pred_folds = []
cm_folds = []

for i, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, y_train = X.iloc[train_index], y.iloc[train_index]
    X_test, y_test = X.iloc[test_index], y.iloc[test_index]

    X_train = np.array(X_train)
    X_test = np.array(X_test)

    y_train = np.array(y_train).astype(int)
    y_test = np.array(y_test).astype(int)

    nm = SMOTE(random_state=42)
    X_res, y_res = nm.fit_resample(X_train, y_train)

    pa1 = {'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.17866332260414736,
           'importance_type': 'split', 'learning_rate': 0.31663818808466854, 'max_depth': 3,
           'min_child_samples': 20, 'min_child_weight': 3.7045345974251696, 'min_split_gain': 0.0,
           'n_estimators': 1191, 'n_jobs': -1, 'num_leaves': 6, 'objective': None, 'random_state': 1,
           'reg_alpha': 2.7808856311910626, 'reg_lambda': 0.43654141243810257, 'silent': True,
           'subsample': 0.7965933438188254, 'subsample_for_bin': 200000, 'subsample_freq': 0, 'max_bin': 323,
           'scale_pos_weight': 3.7269851872292934}
    pa2 = {'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.03379242822507714,
           'importance_type': 'split', 'learning_rate': 0.12613026534842414, 'max_depth': 1,
           'min_child_samples': 16, 'min_child_weight': 4.163032273614103, 'min_split_gain': 0.0,
           'n_estimators': 616, 'n_jobs': -1, 'num_leaves': 7, 'objective': None, 'random_state': 1,
           'reg_alpha': 1.8283478827688406, 'reg_lambda': 0.2069651728126041, 'silent': True,
           'subsample': 0.7005502102578648, 'subsample_for_bin': 200000, 'subsample_freq': 0, 'max_bin': 437,
           'scale_pos_weight': 2.432742203577984}
    pa3 = {'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.41926099496682023,
           'importance_type': 'split', 'learning_rate': 0.29289472226649155, 'max_depth': 9,
           'min_child_samples': 12, 'min_child_weight': 3.564392827408104, 'min_split_gain': 0.0,
           'n_estimators': 1004, 'n_jobs': -1, 'num_leaves': 8, 'objective': None, 'random_state': 1,
           'reg_alpha': 1.5238007376941431, 'reg_lambda': 0.2882531480596396, 'silent': True,
           'subsample': 0.9738277910821249, 'subsample_for_bin': 200000, 'subsample_freq': 0, 'max_bin': 617,
           'scale_pos_weight': 1.5855218660728612}
    params_3fold_res = [pa1, pa2, pa3]

    params_cur = params_3fold_res[i]
    modellgb = LGBMClassifier()
    modellgb.set_params(**params_cur)
    clf_lg = modellgb.fit(X_res, y_res)
    p_lg_train = clf_lg.predict_proba(X_res)  # [:,1]
    p_lg_test = clf_lg.predict_proba(X_test)  # [:,1]
    p_lg_val = clf_lg.predict_proba(X_val)  # [:,1]
    val_folds.append(p_lg_val)
    test_folds.append(p_lg_test)

    l_lg_train = clf_lg.predict(X_res)
    l_lg_test = clf_lg.predict(X_test)
    l_lg_val = clf_lg.predict(X_val)

    f1_train = f1_score(y_res, l_lg_train, average='weighted')
    f1_test = f1_score(y_test, l_lg_test, average='weighted')
    f1_val = f1_score(y_val, l_lg_val, average='weighted')

    true_folds.append(y_test)
    pred_folds.append(l_lg_test)

    lg_auc_train = roc_auc_score(y_res, p_lg_train, multi_class='ovr')
    lg_auc_test = roc_auc_score(y_test, p_lg_test, multi_class='ovr')
    lg_auc_val = roc_auc_score(y_val, p_lg_val, multi_class='ovr')

    cm_test = confusion_matrix(y_test, l_lg_test)
    cm_folds.append(cm_test)

all_folds = np.stack(val_folds)
mean_folds = np.mean(all_folds, axis=0)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
%matplotlib inline

cm = np.sum(np.stack(cm_folds), axis=0)
disp1 = ConfusionMatrixDisplay(confusion_matrix=cm)
disp1.plot()

In [ ]:
all_folds = np.stack(val_folds)
mean_folds = np.mean(all_folds, axis=0)
lg_auc_val = roc_auc_score(y_val, mean_folds, multi_class='ovr')
print('Validation AUC : ', np.round(lg_auc_val, decimals=4))
cm_val = confusion_matrix(y_val, np.argmax(mean_folds, axis=1))
disp1 = ConfusionMatrixDisplay(confusion_matrix=cm_val)
disp1.plot()